In [2]:
import torch

In [4]:
torch.ones(1, 350, 350, dtype=torch.int64).tril()

tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 0, 0],
         [1, 1, 1,  ..., 1, 1, 0],
         [1, 1, 1,  ..., 1, 1, 1]]])

In [4]:
demo = torch.tensor([True, True, True, True, True, False], dtype=torch.int64)
print(demo)

tensor([1, 1, 1, 1, 1, 0])


In [8]:
demo.repeat(6, 1)

tensor([[1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 0]])

In [3]:
mask = torch.triu(torch.ones(size=(1, 350, 350), dtype=torch.int64), diagonal=1) == 0
print(mask)

tensor([[[ True, False, False,  ..., False, False, False],
         [ True,  True, False,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         ...,
         [ True,  True,  True,  ...,  True, False, False],
         [ True,  True,  True,  ...,  True,  True, False],
         [ True,  True,  True,  ...,  True,  True,  True]]])
